In [1]:
from bond import Bond
from curve import Curve
from yieldcalculator import YieldCalculator
from oas import OASModel
from datetime import datetime, date
import requests
import pprint
%load_ext autoreload
%autoreload 2

### Use the Python modules

In [46]:
valueDate = date(2023, 3, 20)

spot_curve = Curve(valueDate)
spot_curve.load_from_csv('./data/treasuryspotcurve.csv')

par_curve = Curve(valueDate)
par_curve.load_from_csv('./data/treasuryparcurve.csv')

bonds = Bond.load_all_bonds()
# price the first 5 bonds
for i in range(5):
    bond = bonds[i]
    bond.calculate_coupon_schedule()
    ytm = YieldCalculator.get_ytm(bond, bond._market_price, valueDate)
    ytc = YieldCalculator.get_ytc(bond, bond._market_price, valueDate)
    ytw = YieldCalculator.get_ytw(bond, bond._market_price, valueDate)
    tenor, trsy_yield = YieldCalculator.get_yield_spread(bond, ytm, par_curve, interpolate=False)
    # model = OASModel()
    # oas = model.Calculate_OAS(bond, spot_curve, valueDate, bond._market_price, 0.001)
    print(f"The bond: {bond}")
    print(f"\tPrice: {bond._market_price}\n\tytm={ytm}\n\tytc={ytc}\n\tytc={ytc}\n\tytw={ytw[0]}\n\tspread={ytm - trsy_yield}")


The bond: 459200AM3 7.000000000000001 10/30/2025
	Price: 105.153
	ytm=0.04972782087703231
	ytc=None
	ytc=None
	ytw=0.04972782087703231
	spread=0.01202782087703231
The bond: 459200KY6 4.75 11/06/2032
	Price: 98.497
	ytm=0.050132419027993404
	ytc=0.050132419027993404
	ytc=0.050132419027993404
	ytw=0.050132419027993404
	spread=0.015432419027993402
The bond: 459200KX8 4.5 01/06/2028
	Price: 98.412
	ytm=0.04934373230976048
	ytc=0.04934373230976048
	ytc=0.04934373230976048
	ytw=0.04934373230976048
	spread=0.013743732309760483
The bond: 459200KZ3 5.1 08/06/2052
	Price: 95.427
	ytm=0.05486920060641127
	ytc=0.05486920060641127
	ytc=0.05486920060641127
	ytw=0.05486920060641127
	spread=0.01836920060641127
The bond: 459200HU8 3.6249999999999996 02/12/2024
	Price: 98.749
	ytm=0.051124337075510944
	ytc=None
	ytc=None
	ytw=0.051124337075510944
	spread=0.0077243370755109425


### Use the REST service
Make sure flast service is started

In [47]:
# Load all bonds
resp = requests.get('http://127.0.0.1:5000/bond')
# print(resp.content)

# Load the bond for CUSIP
resp = requests.get('http://127.0.0.1:5000/bond?cusip=459200AM3')
pprint.pprint(resp.json())


{'Announce': '10/25/1995',
 'Ask Price': 105.153,
 'CUSIP': '459200AM3',
 'Composite Rating': 'A-',
 'Coupon Freq': 2,
 'Coupon Type': 'FIXED',
 'Cpn': 7.000000000000001,
 'Currency': 'USD',
 'Issue Date': '10/30/1995',
 'Issued Amount': 600000000.0,
 'Maturity': '10/30/2025',
 'Maturity Type': 'AT MATURITY',
 'Next Call Date': None,
 'Next Call Price': 100,
 'Recovery': 0.75,
 'Ticker': 'IBM'}


In [48]:
# price a bond given the CUSIP, price and val_date
resp = requests.get('http://127.0.0.1:5000/pricing?cusip=459200AM3&price=105.153&value_date=20230320&oas=0.05')
pprint.pprint(resp.json())

{'CUSIP': '459200AM3',
 'Coupon': 0.07,
 'Maturity': '10/30/2025',
 'OAS': -0.5108559542505597,
 'Price': 105.153,
 'ValueDate': '03/20/2023',
 'jtd': 30.153000000000006,
 'ytc': None,
 'ytm': 0.04972782087703231,
 'ytm to treasury spread': 0.01202782087703231,
 'ytw': 0.04972782087703231,
 'ytw_date': '10/30/2025'}
